In [120]:
import geopandas as gpd
import pandas as pd
import folium
from folium import Element
import ee      # Google Earth Engine API
import geemap
import json
import pycrs
from geopy.geocoders import Nominatim
from osgeo import ogr
import requests
import re
import googlemaps
import time
from math import radians, sin, cos, sqrt, atan2       # Haversine Formula, closest points
import networkx as nx
import time
from IPython.display import HTML
from jinja2 import Template
from folium import plugins
from folium.plugins import GroupedLayerControl
import ee
from datetime import datetime
from branca.element import Template, MacroElement  # for Legend on map
from IPython.display import HTML
import requests
from bs4 import BeautifulSoup
from flask import Flask, render_template, request
import branca.colormap as cm
from dotenv import load_dotenv


from matplotlib.lines import Line2D
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
import rasterio
from rasterio.plot import show
import rioxarray as rxr
import earthpy as et
import earthpy.spatial as es
import earthpy.plot as ep
import fiona
import os
from shapely import wkt
from shapely.geometry import Polygon
from shapely.geometry import box
from shapely.geometry import shape
import matplotlib.pyplot as plt
import contextily as ctx
from osgeo import gdal 
import numpy as np
import glob
import xarray
import xrspatial
import geojson
import osmnx as ox
import scipy
from scipy.spatial import KDTree
from datashader.transfer_functions import shade
import richdem as rd                                    # calculate slope
from rioxarray.merge import merge_arrays                # merge raster arrays together 
from osgeo import gdal                                 # to merge rasters together
import gemgis as gg


### CHANGE DIRECTORY

In [123]:
# Change directory
base_dir_path = os.getcwd() # Use getcwd() for Jupyter Notebook
os.chdir(base_dir_path)

### OPENSTREET MAPS DATA

In [4]:
# Use osmnx to find all EMS locations in SD County 

def get_street_network(county):
    g = ox.graph_from_place(county, network_type = 'drive')
    return g

def show_street_network(county_street_network):
    ox.plot_graph(county_street_network)

# Load all amenities
def load_all_hospitals(county):
    ems_amenities = {'amenity': ['hospital']}
    return ox.geometries_from_place(county,ems_amenities)


def load_all_firestations(county):
    fd_amenities = {'amenity': ['fire_station']}
    df = ox.geometries_from_place(county,fd_amenities)

    locations = []
    for index, fd in df.iterrows():
        if fd['geometry'].type== 'Point':
            latitude = float(fd.geometry.y)
            longitude = float(fd.geometry.x)
            locations.append([longitude, latitude])
        else:
            centroid = fd.geometry.centroid  # Get centroid of Polygon
            latitude = float(centroid.y)
            longitude = float(centroid.x)
            locations.append([longitude, latitude])
    return locations
        

def filter_emergency_nodes(ems_geometry):
    return ems_geometry[ems_geometry.geometry.type == 'Point']

def get_nearest_nodes(county_street_network, filtered_emergency_nodes):
    # Get the EMS nodes nearest to each EMS point
    ems_nodes = [ox.distance.nearest_nodes(county_street_network, point.x, point.y) for point in filtered_emergency_nodes.geometry]
    return ems_nodes


    
   
# Get the graph in a format that NetworkX can work with
def add_edge_speed(county_street_network):
    return ox.speed.add_edge_speeds(county_street_network)

def add_travel_times(county_street_network_edge_speed):
    return ox.speed.add_edge_travel_times(county_street_network_edge_speed)


  
    


## CENSUS DATA API

### Census Population

In [5]:
def get_census_population_data(county_fip):
    
    '''
    Makes API call to census.gov to obtain American Community Survey dataset. This dataset reports the total population for each block group
    within the county of choice. For even finer granularity, we may use the decennial dataset url to obtain block data. 

    Args
    ----
    county_fip (str): returns the county FIP code 

    Returns
    ------
    Census dataframe from census json response data. Creates new column ID that concats tract + block group codes, used to join TIGER boundary df.
    
    '''
    base_url = 'https://api.census.gov/data/2022/acs/acs5' # American Community Survey for the block group level (coarser granularity than block)
    #base_url = "https://api.census.gov/data/2010/sf1" decennial data at the block level (finest granularity)
    params = {
        "get": "B01003_001E", # Returns total population. For Decennial Census dataset, use  "P001001" (most recent data is from 2010 for Decennial)
        "for": "block group:*",
        "in": f"state:06 county:{county_fip}",  # 06 is the FIPS code for California, 113 is the FIPS code for Yolo County
        "key": "208a838e2db7e2bdff8bc7fff18b0c36ae890abd"
    }
    
    headers = {
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36'
          }
    
    #API_key = '208a838e2db7e2bdff8bc7fff18b0c36ae890abd'
    
    response = requests.get(base_url, params=params, headers=headers)

    census_response = response.json()
    census_df = (pd
                .DataFrame(census_response)
                .rename(columns = {0: 'Total_Population', 1: 'State', 2: 'County', 3: 'Tract', 4: 'block_group'})
                .iloc[1:]
                .assign(** {'Total_Population': lambda x: x['Total_Population'].astype(int)})
                .assign(Tract_BG = lambda x:x['Tract'] + x['block_group']) # creates new column with unique ID used to join boundary df 
                .reset_index(drop=True)
                )
                 
    return census_df


### Census Block Groups

In [6]:
def get_census_block_group_shapefile(county_fip):

    '''
    Parameters
    ----------
    county_fip (str): the county FIP code

    Returns
    -------
    TIGER block group boundaries of county of interest. Creates new column that concatenates tract + block group id codes to produce a unique index.
    This will be used to join on unique index of census population df.
    
    '''
    
    # URL for the TIGER/Line Shapefiles for census tracts
    block_groups_url = "https://www2.census.gov/geo/tiger/TIGER2021/BG/tl_2021_06_bg.zip"
    
    # Read the shapefile directly from the URL
    gdf = (gpd
           .read_file(block_groups_url)
           .query(f" COUNTYFP == '{county_fip}'")
           .assign(Tract_BG = lambda x : x['TRACTCE'] + x['BLKGRPCE']).sort_values(by = ['TRACTCE', 'BLKGRPCE']).reset_index(drop=True) # concatenates tract code and block group code to form unique code that will be used to join census population dataframe
           
)
    
    # Filter for Yolo County (FIPS code for Yolo County is 113)
    #block_groups = gdf[gdf['COUNTYFP'] == county_fip]

        
    # Optionally, save to a local file
    #yolo_tracts.to_file("yolo_tracts.shp")

    return gdf

def merge_shapefile_population_dfs(population_df, block_group_df, crs):

    ''' 
    Merges Census Block Group geometries with Census Population. 
    
    '''
    census_merged = block_group_df.merge(population_df, on = 'Tract_BG').drop_duplicates()
    return census_merged.to_crs(crs)
     
def calculate_population_within_isochrone(census_gdf, isochrone_gdf, crs):

    
    census_gdf['total_block_group_area'] = census_gdf.to_crs('EPSG:3857').area

    # Combines all isochrone geometries into one single geometry - 
    union_isochrone = isochrone_gdf.to_crs('EPSG:3857').unary_union 
    
    intersection = (census_gdf
                    .to_crs('EPSG:3857') # Projected CRS in meters. Gives coordinates on a flat 2-D surface
                    .assign(total_block_group_area = lambda x: x.area)
                    .assign(intersection_area = lambda x: x.intersection(union_isochrone).area)
                    .assign(intersection_area_ratio = lambda x: x['intersection_area']/x['total_block_group_area'])
                    .assign(population_within_isochrone = lambda x: round(x['Total_Population'] * x['intersection_area_ratio'],0))
                    .assign(population_percentage = lambda x:  round(x['intersection_area_ratio']*100, 0))
                    .to_crs(crs) # Reproject back to geometric coordinate system 
    )

   
    #intersection.merge(census_gdf, on = 'GEOID')
    
    return intersection

def plot_census_block_group_pop_on_matplotlib_axis(merged_shapefile_pop, ax, alpha):
    return merged_shapefile_pop.plot(ax = ax, column = 'Total_Population', alpha = alpha)


## CREATE ISOCHRONE MAP 

### CREATE NETWORK GRAPHS 

In [7]:

def graph_from_place(place_name):
    '''
    Generates street network graph for place of interest. OpenStreetMap for the specified place, 
    retrieves the street network data, and constructs a NetworkX graph object. Nodes in the graph 
    represent intersections or dead ends, and edges represent the streets connecting these nodes. 
    '''
    return ox.graph_from_place(place_name, network_type= 'drive', retain_all= True, truncate_by_edge= True)

def add_street_speeds(place_name_network):
    '''
    Estimates speed for each edge on the graph based on the type of street (i.e. residential, primary, secondary)
    '''
    return ox.speed.add_edge_speeds(place_name_network)

def add_street_travel_times(place_name_network):
    ''' 
    Calculates travel times by (length of edges)/ (speed of edges). Need to calculate edge speeds first. 
    '''
    return ox.speed.add_edge_travel_times(place_name_network)



### LOAD FD DATA

#### MAP BOX API

In [ ]:
# Mapbox token
access_token = os.get_env('ACCESS_TOKEN')

# Define a function to geocode an address using Mapbox Geocoding API
def geocode_address_mapbox(address):
    ''' 
    Uses mapbox api to transform addresses that contain street address, county, zip code and state. An api call is made to mapbox.com, 
    if the response is successful, and a JSON object is returned. Within the features object of the JSON, there is a geometry object that 
    contains information on the coordinates in the form : [longitude, latitude]. 

        JSON object returned, example:   {
                                              "type": "FeatureCollection",
                                              "features": [
                                                {
                                                  "type": "Feature",
                                                  "geometry": {
                                                    "type": "Point",
                                                    "coordinates": [longitude, latitude]
                                                  },
                                                  "properties": {
                                                    "name": "Place Name",
                                                    "address": "123 Main St",
                                                    "city": "City",
                                                    "state": "State",
                                                    "country": "Country",
                                                    "postcode": "12345"
                                                    // Other relevant properties
                                                  }

    
    '''
    
    # Construct the query URL with the address
    query = address
    url = f'https://api.mapbox.com/geocoding/v5/mapbox.places/{query}.json?access_token={access_token}'
    
    # Make a GET request to the API
    response = requests.get(url)
    
    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()
        
        # Check if any features were found
        if data['features']:
            # Extract the latitude and longitude
            longitude, latitude = data['features'][0]['geometry']['coordinates']
            return latitude, longitude
        else:
            # No results found for the address
            return None, None
    else:
        # Request failed
        return None, None

# All California firestations from the registry at www.usfa.fema.gov. Geocoding using .apply(geocode_address_mapbox)
def firestations_to_df(CA_firestations_txt):

    '''
    If the PO Box is NaN, the full concatenated addresses is "NaN", therefore this function will process the dataframe accordingly
    '''
    station_df = pd.read_csv(CA_firestations_txt)
    
    for index, firestation in station_df.iterrows():
        if pd.notna(firestation['Mail PO box']):
            firestations_registry = station_df.assign(Full_Address = lambda x : x['HQ addr1'] + ',' + ' '+ x['HQ city'] + ',' + ' '+ x['HQ state'] + ',' + ' '+ x['Mail PO box'])
        else:
             firestations_registry = station_df.assign(Full_Address = lambda x : x['HQ addr1'] + ',' + ' '+ x['HQ city'] + ',' + ' ' + x['HQ state'])

    geocoded = (firestations_registry
                .assign(Coordinates_Firestation = lambda x:x['Full_Address'].apply(geocode_address_mapbox)) # Geocoded coordinates 
                .to_csv('geocoded_CA_firestations.csv') # Saves as .csv
               )
    return pd.read_csv(geocoded)

     


In [97]:

def find_nearest_nodes_to_firestations(place_name_network, fire_stations_df):
    ''' 
    Finds the closest nodes to the firestations by longitude and latitude. This node will then be used in the
      network analysis as the starting point for all node calculations
    '''
    return [ox.distance.nearest_nodes(place_name_network, lon, lat) for lon, lat in fire_stations_df]

def load_all_fds_within_county(county_gdf, crs):
    
    bounds = county_gdf.total_bounds
    
    csv_geocoded_fd_all = os.path.join(base_dir_path,  'Data_Shapefiles', 'Firestations', 'geocoded_CA_firestations.csv')

    #firestations_to_df(fd_txt)
    firestations_df = (pd
                    .read_csv(csv_geocoded_fd_all)
                    .assign(
                        Latitude=lambda x: x['Coordinates_Firestation'].apply(lambda coord: None if coord.strip() == '(None, None)' else float(coord.strip('()').split(',')[0])),             
                        Longitude=lambda x: x['Coordinates_Firestation'].apply(lambda coord: None if coord.strip() == '(None, None)' else float(coord.strip('()').split(',')[1]))) # split ['Coordinates_Firestation'] on the " , " and make new columns for Lat/Lon. Make these floats.
                    .replace('', np.nan) # Handles any blank coordinates that were improperly loaded
    )
    firestations_df = firestations_df.dropna(subset=['Latitude', 'Longitude'])

    # convert to GeoDataframe, and set the geometry to the longitude and latitude of the firestations. Set the crs        
    firestations_gdf = gpd.GeoDataFrame(
                        firestations_df,
                        geometry=gpd.points_from_xy(firestations_df['Longitude'], firestations_df['Latitude']),
                        crs=crs
    )

    # Use spatial join to select stations within county 
    return gpd.sjoin(firestations_gdf, county_gdf, how='inner', op='within')


def extract_all_firestations_coords_within_county_bounds(county_gdf, firestation_gdf):
    ''' 
    Returns
    --------
    list of [Lon, Lat] coordinate pairs. These coordinate pairs will be used to find the closest nodes.

    '''
    #fd_amenities = {'amenity': ['fire_station']}
    #df = ox.geometries_from_place(county,fd_amenities)


    fd_coordinates = []
    for index, fd in firestation_gdf.iterrows():
        #if (bounds[1] <= fd['Latitude']<= bounds[3]) & (bounds[0] <= fd['Longitude'] <= bounds[2]):
        fd_coordinates.append([fd['Longitude'], fd['Latitude']])

    return fd_coordinates


def add_firestations_to_map(firestations_df, folium_map):
    
    firestations_layer = folium.FeatureGroup(name = 'Fire Stations')
    
    for row, fd in firestations_df.iterrows():
            folium.Marker(location = [fd['Latitude'], fd['Longitude']], 
                            icon=folium.Icon(icon='fire', prefix = 'fa', color='red', name = 'Firestations'),
                            tooltip = fd['Fire dept name']).add_to(firestations_layer)
            
    firestations_layer.add_to(folium_map)
    return folium_map


### CREATE ISOCHRONES

In [9]:

def make_iso_polys(place_name_network, fire_station_nodes, travel_times):
    ''' 
    Creates convex hull

    Arguments
    ---------
    fire_station_nodes : List of center nodes from which the radius of the isochrone is calculated. In this case, these
                    are the firestation nodes. These serve as the center points.
    travel_times (int) : the total travel time (in seconds) from the center node

    Returns
    -------- 
    List of isochrone polygons representing the area around each firestation that is within __ minutes travel time.
    
    '''
    isochrone_polys = []

    for center_node in fire_station_nodes:
        subgraph = nx.ego_graph(place_name_network, center_node, radius=travel_times, distance='travel_time') # Graph of all nodes within a certain trip time (radius) to the ego node (center node)
        node_points = [(place_name_network.nodes[node]['x'], place_name_network.nodes[node]['y']) for node in subgraph.nodes if 'x' in place_name_network.nodes[node] and 'y' in place_name_network.nodes[node]]
        
        if node_points:  # Ensure node_points is not empty
            try:
                gdf_nodes = gpd.GeoDataFrame(crs=4269, geometry=gpd.points_from_xy([point[0] for point in node_points], [point[1] for point in node_points]))
                convex_hull = gdf_nodes.unary_union.convex_hull
                isochrone_polys.append(convex_hull)
            except TypeError as e:
                print(f"Error creating GeoSeries: {e}")

    return isochrone_polys

def get_isochrones_list(place_name_network, fire_station_nodes, travel_times):
    ''' 
    
    '''
    isochrones = []
    for node in fire_station_nodes:
        isochrones.extend(make_iso_polys(place_name_network, fire_station_nodes, travel_times))
    return isochrones 

def isochrone_df_geometry(isochrone_list, crs):
    ''' 
    Returns
    -------
    dataframe of the isochrones with one column, 'geometry'
    '''
    return gpd.GeoDataFrame(crs = crs, geometry=isochrone_list)



## ADD ELEMENTS TO FOLIUM MAP

### ISOCHRONE LAYERS

In [10]:
# Functions to Add Elements to Map
def add_isochrone_layer_to_map(isochrone_list, travel_minutes, show):
    

    isochrone_layer = folium.FeatureGroup(name = f"{travel_minutes} Minutes", control = True, show = show)
 
    for isochrone in isochrone_list:
        if isinstance(isochrone, Polygon):  # Ensure it's a Polygon
            folium.GeoJson(data=isochrone, 
                        style_function=lambda x: {
                                    'fillColor': 'red', 
                                    'color': 'red', 
                                    'weight': .05, 
                                    'fillOpacity': 0.05}).add_to(isochrone_layer)
            
    
    return isochrone_layer

def add_grouped_layers_legend(folium_map):

    # Create a dictionary for the grouped layers
    grouped_layers = {
        'Travel Times from Fire Department': [
            isochrone_layer_4,
            isochrone_layer_6,
            isochrone_layer_8,
            isochrone_layer_10
        ]
    }
    # Add travel time isochrones into a Grouped Layer
    GroupedLayerControl(groups= grouped_layers, 
                        exclusive_groups= False,#['Areas within Travel Time from Fire Department'],
                        position = 'bottomleft', 
                        collapsed = False).add_to(folium_map)



### NETWORK GRAPHS AND PERIMETERS

In [11]:
def add_network_to_folium_map(place_name_network, folium_map, color, opacity):
    '''
    Converts the graph edges to GeoJSON format and adds them to a Folium map.
    '''
    geodata = ox.graph_to_gdfs(place_name_network, nodes=False, edges=True, fill_edge_geometry=True)  # Convert edges to GeoDataFrame
    geojson = geodata.to_json()  # Convert GeoDataFrame to GeoJSON
    
    folium.GeoJson(
        geojson,
        name='Street Network',
        style_function=lambda feature: {
            'color': color,
            'weight': 0.3,
            'opacity': opacity,
        }
    ).add_to(folium_map)

def get_county_bounds(county_gdf):
    ''' 
    Calculate the bounding box of the county.
    '''
    bounds = county_gdf.total_bounds  # Returns [minx, miny, maxx, maxy]
    southwest = [bounds[1], bounds[0]]  # [miny, minx]
    northeast = [bounds[3], bounds[2]]  # [maxy, maxx]
    return [southwest, northeast]

def add_county_border_to_folium_map(place_name, folium_map, crs):

    county_json = (ox
              .geocode_to_gdf(place_name)
              .to_crs(crs)
              .to_json()
            )
    folium.GeoJson(data = county_json,
                    name = 'County Border',
                    style_function = lambda x : {
                                            'fillColor' : 'none',
                                            'color' : 'black', 
                                            'weight' : 2
                                            }).add_to(folium_map)
    return m

def create_color_map(merged_pop_df):
    min_population = merged_pop_df['Total_Population'].min()
    max_population = merged_pop_df['Total_Population'].max()
    
    
    colormap = cm.LinearColormap(
        vmin=min_population,
        vmax=max_population,
        colors=['blue', 'green', 'yellow', 'orange'],
        #colors = ['#ccebc5', '#7bccc4', '#2b8cbe', '#084081'],
        caption='Total Population by Census Block Group'
    )
    
    return colormap


### CENSUS DATA AND TOOLTIP

In [12]:
def create_combined_tooltip(row, populations, percentages):
    tooltip_text = (
        f"<b>TRACT:</b> {row['Tract']} <br>"
        f"<b>BLOCK GROUP:</b> {row['block_group']} <br>"
        f"<b>TOTAL POPULATION:</b> {row['Total_Population']} <br>"
        f"<b>ESTIMATED POPULATION WITHIN:</b> <br>"
    )
    for travel_time in populations.keys():
        population = populations.get(travel_time, 'N/A')
        pop_percent = percentages.get(travel_time, 'N/A')
        tooltip_text += f"<b>{travel_time} Minutes:</b> {int(population)} ({int(pop_percent)}%) <br>"
    return folium.Tooltip(tooltip_text)

# Function to add census block group data with combined tooltips to the map
def add_census_block_group_data_to_folium_map(census_pop_dfs, travel_minutes, colormap, folium_map):
    # Create dictionaries to store population within isochrone and percentages for each block group
    block_group_populations = {}
    block_group_population_percentages = {}
    feature_layer = folium.FeatureGroup(name='Census Block Groups')

    # Define a color scale for population
    def get_color(population, colormap):
        return colormap(population)

    # Iterate through each GeoDataFrame and store populations and percentages in the dictionaries
    for census_pop_df, travel_time in zip(census_pop_dfs, travel_minutes):
        for _, row in census_pop_df.iterrows():
            block_group = row['GEOID']
            if block_group not in block_group_populations:
                block_group_populations[block_group] = {}
                block_group_population_percentages[block_group] = {}
            block_group_populations[block_group][travel_time] = row['population_within_isochrone']
            block_group_population_percentages[block_group][travel_time] = row['population_percentage']
            

    # Add GeoJson layers with combined tooltips to the map
    for _, row in census_pop_dfs[0].iterrows():  # Use the first dataframe as a reference
        block_group = row['GEOID']
        if block_group in block_group_populations:
            populations = block_group_populations[block_group]
            pop_percents = block_group_population_percentages[block_group]
            geojson = folium.GeoJson(
                data=row['geometry'].__geo_interface__,
                style_function=lambda feature, row=row: {
                    'fillColor': get_color(row['Total_Population'], colormap),
                    'color': 'black',
                    'weight': 0.75,
                    'fillOpacity': 0.3
                },
                tooltip=create_combined_tooltip(row, populations, pop_percents)
            )
            geojson.add_to(feature_layer)

    feature_layer.add_to(folium_map)




### TITLE AND MAP INFO BUTTON 

In [118]:
def add_map_title(folium_map, title):
    # Add Title
    title_html = f'''
    <div style="position: absolute; top: 2%; left: 25%; transform: translateX(-50%); z-index: 1000; 
                background-color: white; padding: 5px 10px; border-radius: .25px; box-shadow: 0 0 10px rgba(0,0,0,0.5);
                line-height:normal;">
                
        <h3 style="margin: 0; font-size: 18px; font-family: 'Didot';">{title}</h3>
    </div>
    '''
    title_element = Element(title_html)
    folium_map.get_root().html.add_child(title_element)

def render_html(element_to_render):
    return element_to_render.get_root().render()

def add_map_information_button(map):
    
    """
    Returns html to add a map information button element

    """
    
    map_info_button =   """
    <style>
    .collapsible {
        background-color: #777;
        color: white;
        cursor: pointer;
        padding: 10px;
        width: 235px;
        border: none;
        text-align: left;
        outline: none;
        font-size: 15px;
        position: fixed; 
        top: 70%; 
        left: 1%;
        z-index:9999;
    }

    .active, .collapsible:hover {
        background-color: #555;
    }

    .content {
        padding: 0 18px;
        display: none;
        overflow: hidden;
        background-color: #f1f1f1;
        border: 2px solid grey;
        width: 300px;
        position: fixed; 
        top: 30%; 
        left: 1%;
        font-size: 10px;
        z-index:9999;
        max-height: 313px;
        overflow-y: auto;
    }
    </style>

    <button class="collapsible">Map Information & Data Sources</button>
    <div class="content">
        <h4>Map Information</h4>
        <p><b>ABOUT THIS ISOCHRONE MAP</b> <br> Travel time to an emergency is an important factor in deciding where to locate fire departments within counties. The ideal response time for a fire department to an emergency is within 4 minutes in large metro areas -- however, the travel time usually increases in more rural communities (National Fire Data Center).</br><br> This map aims to show the area that a fire department can travel to within 4 minutes, 6 minutes, 8 minutes and 10 minutes in Calaveras County, a county that has already seen large fires and emergencies in 2024.</br><br> The population within each Census Block Group of the county is also given. The combination of drive times and population data can hopefully highlight areas that are underserved, or where an additional firehouse would increase coverage.</br></p>
        <p><b>ABOUT THE DATA AND METHODS</b> <br><b>Network Data and Analysis</b> The OSMnx open-source library was used to conduct the network analysis by using <i>OpenStreetMaps</i> to query CA county spatial data and construct node graphs of the county of interest. Edge speeds and travel times within the node graph of the county were also calculated using OSMnx.</br>
        <br><b>Census Data</b>: Data was fetched via an API request to the American Community Survey endpoint -- the block group dataset was queried to maintain a high level of granularity.</br>
        <br><b>Calculation of Population within Travel Times:</b> To estimate the population within the travel times, the area for each block group was calculated. Next, all isochrone geometries were combined to form a single geometry. The area of intersection of the isochrone geometry and each block group was then calculated. To estimate the population within the travel time for each block group, the ratio is multiplied by Total Population</br>
        <br><b> Fire Station Data:</b> The fire station location data may be outdated or not current and is not necessarily a complete list of fire stations in California. </br>
        </p>
    
        <h4>Data Sources</h4>
        <ul>
            <li><a href="https://www.census.gov/en.html" target="_blank">US Census Bureau</a></li>
            <li><a href = "https://apps.usfa.fema.gov/registry/" target="_blank">US Fire Administration</a></li>
        </ul>

    </div>
    """
    references_control = Element(map_info_button)
    map.get_root().html.add_child(references_control)
    return map

def enable_info_button_interactivity():
    """
    Returns javascript code to enable show/hide interactivity
    """

    return """
    <script>
    document.addEventListener("DOMContentLoaded", function() {
        var coll = document.getElementsByClassName("collapsible");
        for (var i = 0; i < coll.length; i++) {
            coll[i].addEventListener("click", function() {
                this.classList.toggle("active");
                var content = this.nextElementSibling;
                if (content.style.display === "block") {
                    content.style.display = "none";
                } else {
                    content.style.display = "block";
                }
            });
        }
    });
    </script>
    """


In [119]:
# Change these variables to return different maps 
place_name = "Calaveras County, California, USA"
county_fip = '009'
crs = 'EPSG:4269'

# -------------------------------- ADD CENSUS DATA --------------------------
# Census Pop. Data and Shapefile
place_pop = get_census_population_data(county_fip)
place_block_group_shp = get_census_block_group_shapefile(county_fip)
county_gdf = merge_shapefile_population_dfs(place_pop, place_block_group_shp, crs= crs)



# -------------------------------------- INSTANTIATE MAP --------------------------
m = folium.Map( tiles = 'CartoDB.Positron')
bounds = get_county_bounds(merge_shapefile_population_dfs(place_pop, place_block_group_shp, crs = crs)) # Calculate the bounding box of the county and fit the map to it
m.fit_bounds(bounds)




# ------------------------------------- CREATE STREET NETWORK / NODE GRAPHS ---------------------------

place_name_street_network = graph_from_place(place_name)
add_street_speeds(place_name_street_network)
add_street_travel_times(place_name_street_network)

# --------------------------------------- GET FIRESTATION COORDS AND NEAREST NODES --------------------------- 
firestation_gdf = load_all_fds_within_county(county_gdf, crs= crs) # From US Fire Administration.com
fire_stations_coords = extract_all_firestations_coords_within_county_bounds(county_gdf = county_gdf, firestation_gdf = firestation_gdf)
nearest_nodes = find_nearest_nodes_to_firestations(place_name_street_network, fire_stations_coords) # These are the nodes that represent firestation locations. To be used as starting node for isochrone calc


# ------------------------------------- ESTABLISH TRAVEL TIMES AND CREATE ISOCHRONES USING FD COORDS AS CENTRAL NODES ---------------------------------------
travel_times_4 = 4*60 # 8 minutes
travel_times_6 = 6*60 # 8 minutes
travel_times_8 = 8*60 # 8 minutes
travel_times_10 = 10*60 # 8 minutes

# Get list of all isochrone geometries for the particular travel times. 
isochrone_list_4 = get_isochrones_list(place_name_network= place_name_street_network, fire_station_nodes= nearest_nodes, travel_times= travel_times_4)
isochrone_list_6 = get_isochrones_list(place_name_network= place_name_street_network, fire_station_nodes= nearest_nodes, travel_times= travel_times_6)
isochrone_list_8 = get_isochrones_list(place_name_network= place_name_street_network, fire_station_nodes= nearest_nodes, travel_times= travel_times_8)
isochrone_list_10 = get_isochrones_list(place_name_network= place_name_street_network, fire_station_nodes= nearest_nodes, travel_times= travel_times_10)


# GeoDataframe of Isochrone geometries for each travel time. This will be used to calculate the population within each isochrone.
isochrone_gdf_4 = isochrone_df_geometry(isochrone_list_4, crs = crs)
isochrone_gdf_6= isochrone_df_geometry(isochrone_list_6, crs = crs)
isochrone_gdf_8 = isochrone_df_geometry(isochrone_list_8, crs = crs)
isochrone_gdf_10 = isochrone_df_geometry(isochrone_list_10, crs = crs)
isochrone_gdf_list = [isochrone_gdf_4, isochrone_gdf_6, isochrone_gdf_8, isochrone_gdf_10]





# Color Map for Block Group Choropleth
colormap = create_color_map(county_gdf)
colormap.add_to(m)

pop_within_isochrone_df_4 = calculate_population_within_isochrone(county_gdf, isochrone_gdf_4, crs = crs)
pop_within_isochrone_df_6 = calculate_population_within_isochrone(county_gdf, isochrone_gdf_6, crs = crs)
pop_within_isochrone_df_8 = calculate_population_within_isochrone(county_gdf, isochrone_gdf_8, crs = crs)
pop_within_isochrone_df_10 = calculate_population_within_isochrone(county_gdf, isochrone_gdf_10, crs = crs)
census_pop_dfs = [pop_within_isochrone_df_4, pop_within_isochrone_df_6, pop_within_isochrone_df_8, pop_within_isochrone_df_10] # Combine all GeoDataFrames into a list


# --------------------------------- ADD DATA TO FOLIUM MAP -----------------------------


# Add Census Data
travel_minutes = ['4', '6', '8', '10']  # Corresponding travel times for each GeoDataFrame
add_census_block_group_data_to_folium_map(census_pop_dfs, travel_minutes, colormap, m)


# Add Isochrone Layers
isochrone_layer_4 = add_isochrone_layer_to_map(isochrone_list_4, '4', show = False).add_to(m)
isochrone_layer_6 = add_isochrone_layer_to_map(isochrone_list_6, '6', show = False).add_to(m)
isochrone_layer_8 = add_isochrone_layer_to_map(isochrone_list_8, '8', show = True).add_to(m)
isochrone_layer_10 = add_isochrone_layer_to_map(isochrone_list_10, '10', show = False).add_to(m)
add_grouped_layers_legend(m) # Add Isochrone Travel Times Legend


add_network_to_folium_map(place_name_street_network, m, color = '#525252', opacity= 0.65)
add_firestations_to_map(firestation_gdf, m)
add_county_border_to_folium_map(place_name, m, crs= crs)
title = f"<p><b>{place_name.rstrip(', USA').upper()}</b></p><br><i>Areas within 4-10 Minutes of travel time</br><br>to a fire station</i></br>"
add_map_title(m, title= title)
add_map_information_button(m)
folium.LayerControl(position = 'topleft').add_to(m)


# -------------------------------------- SAVE MAP -------------------------------
save_file_name = f"{place_name.rstrip(', California, USA')}_network"
m.save(f"{save_file_name}.html")

# Read the saved HTML file
with open(f"{save_file_name}.html", 'r') as f:
    m_html = f.read()

# Inject the JavaScript for interactivity
map_button_interactivity = enable_info_button_interactivity()
m_interactive = m_html.replace('</body>', map_button_interactivity + '</body>')

# Save the modified HTML to a new file
with open(f"{save_file_name}_interactive.html", 'w') as f:
    f.write(m_interactive)
















/Users/andresabala/anaconda3/envs/gis_env/lib/python3.12/site-packages/shapely/set_operations.py:131: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
/Users/andresabala/anaconda3/envs/gis_env/lib/python3.12/site-packages/shapely/set_operations.py:131: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
